In [1]:
import pandas as pd

df = pd.read_parquet("../data/processed/sapporo_density.parquet")
df.head(20)

,d,t,x,y,count
0,0,0,23,153,23
1,0,0,24,153,20
2,0,0,17,163,8
3,0,0,18,156,7
4,0,0,22,151,7
5,0,0,22,158,7
6,0,0,23,152,7
7,0,0,24,155,7
8,0,0,26,151,7
9,0,0,29,138,7
